Compare simulation with experiment

***
#### Imports

In [ ]:
import array
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal
from rslaser.utils import srwl_uti_data as srwutil
from rsmath import lct as rslct

import scipy.constants as const
from scipy.ndimage import gaussian_filter
from scipy import interpolate
from scipy import special

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

#### File Data

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')

files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_pump_off.txt'), #_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_pump_off.txt'), #_off_peak_timing.txt'), #_pump_off.txt')
)

### Pulse Init

In [ ]:
wp = 1.64e-3#1.2e-3 /1.18#1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = 10,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / math.sqrt(2.),
    sigx_waist  = wp,
    sigy_waist  = wp,
)

thisPulse = pulse.LaserPulse(params, files)

In [ ]:
# Plot initial wavefront
wfr_exp_init = thisPulse.slice[0].wfr

x_exp_init=np.linspace(wfr_exp_init.mesh.xStart,wfr_exp_init.mesh.xFin,wfr_exp_init.mesh.nx)
y_exp_init=np.linspace(wfr_exp_init.mesh.yStart,wfr_exp_init.mesh.yFin,wfr_exp_init.mesh.ny)

e_total = thisPulse.extract_total_2d_elec_fields()
intensity_2D_exp_init = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

fig = plt.figure()#figsize=(6,3.6))
ax = fig.gca()
plt.pcolormesh(y_exp_init*(1e3), x_exp_init*(1e3), intensity_2D_exp_init, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (pump_off)')

In [ ]:
# initial phase
phase_exp_init = srwlib.array('d', [0]*wfr_exp_init.mesh.nx*wfr_exp_init.mesh.ny) # "flat" array to take 2D data
srwl.CalcIntFromElecField(phase_exp_init, wfr_exp_init, 0, 4, 3, wfr_exp_init.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping data from flat to 2D array
phase_2d_exp_init = np.unwrap(np.unwrap(np.array(phase_exp_init).reshape((wfr_exp_init.mesh.nx, wfr_exp_init.mesh.ny), order='C').astype(np.float64),axis=0),axis=1)

fig = plt.figure()
plt.pcolormesh(y_exp_init*(1e3), x_exp_init*(1e3), phase_2d_exp_init, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
plt.xlabel(r'Horizontal Position [mm]')
plt.ylabel(r'Vertical Position [mm]')
plt.title('Phase (pump off)')

***
### Crystal Init

In [ ]:
z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([29.0, 19.0, 11.5, 6.7, 4.0, 2.5])
plot_fit = interpolate.splrep(z, n2_plot)

n2_plot2 = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot2_fit = interpolate.splrep(z, n2_plot2)

num_slices = 10
n0 = 1.76
n2 = 16.0
L_cryst = 0.025
# gamma = np.sqrt(n2/n0)

new_z = (L_cryst /num_slices) *(np.arange(num_slices)+0.5)
print(new_z[0])

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_slices,
    # l_scale     = 0.001,
    n0          = [n0 for _ in range(num_slices)],
    n2          = interpolate.splev(new_z, plot2_fit).tolist(), #[n2 for _ in range(num_slices)],#
    pump_energy = 0.035,  # [J]
    pump_waist  = wp, # [m]
    # A           = np.cos(gamma*L_cryst),
    # B           = (1/gamma)*np.sin(gamma*L_cryst),
    # C           = -gamma*np.sin(gamma*L_cryst),
    # D           = np.cos(gamma*L_cryst)
)

# test_z = np.linspace(0.0,2.5,100)/100.0
# fig = plt.figure()
# plt.plot(test_z*(1e2), interpolate.splev(test_z, plot_fit), 'k', label='Old Fit')
# plt.plot(test_z*(1e2), interpolate.splev(test_z, plot2_fit), 'r', label='New Fit')
# plt.legend()
# plt.ylim(0,40)
# plt.xlim(0,2.5)
# plt.ylabel(r'n2 [1/m$^2$]')#[mm]')
# plt.xlabel(r'z [cm]')#[mm]')
# plt.title('Simulation n2 Variation')

e_crystal = crystal.Crystal(crystal_params)

***
## Check scale factor and resampling

In [ ]:
# # photon_e_ev = laser_pulse.photon_e_ev

# ##Convert energy to wavelength
# hc_ev_um = 1.23984198   # hc [eV*um]
# phLambda = hc_ev_um / params.photon_e_ev * 1e-6 # wavelength corresponding to photon_e_ev in meters
# print("Wavelength corresponding to %g keV: %g microns" %(params.photon_e_ev * 1e-3, phLambda / 1e-6))

# # rescale ABCD matrix with wavelength and scale factor for use in LCT algorithm
# A = 0.99765495
# B = 1.41975385 * phLambda / (crystal_params.l_scale**2)
# C = -0.0023775 / phLambda * (crystal_params.l_scale**2)
# D = 0.99896716
# abcd_mat_cryst = np.array([[ A,  B  ],
#             [ C, D ]])
# abcd_decomp = rslct.lct_decomposition(abcd_mat_cryst)
# print(abcd_decomp)

In [ ]:
# prop_mesh_den_pred = abcd_decomp[2][1] * abcd_decomp[5][1] * np.shape(phase_2d_exp_init)[0]
# print('predicted propagated mesh density: %g, %g' %(prop_mesh_den_pred, prop_mesh_den_pred))

***
## Propagate through crystal once

Requires two separate propagations (one at n2=max, one at n2=0) that are then combined

Phase requires np.unwrap to be called once for each axis

In [ ]:
prop_type = 'n0n2_srw' #'gain_calc' #'abcd_lct' #'n0n2_lct' #'n0n2_srw'
gain = 1
radial_n2 = 1

LP = pulse.LaserPulse(params, files)
nslices_pulse = len(LP.slice)
e_crystal = crystal.Crystal(crystal_params)
nslices_crystal = len(e_crystal.slice)

n_photons_before = 0
for laser_index_i in np.arange(LP.nslice):
        n_photons_before += np.sum(LP.slice[laser_index_i].n_photons_2d.mesh)

print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
LP = e_crystal.propagate(LP, prop_type, gain, radial_n2)

In [ ]:
# instantiate a drift
L_drift = 0.50  # [m]
e_drift = drift.Drift(L_drift)
LP = e_drift.propagate(LP, 'default')

wfr_sim_final=LP.slice[0].wfr
x_sim_final=np.linspace(wfr_sim_final.mesh.xStart,wfr_sim_final.mesh.xFin,wfr_sim_final.mesh.nx)
y_sim_final=np.linspace(wfr_sim_final.mesh.yStart,wfr_sim_final.mesh.yFin,wfr_sim_final.mesh.ny)

# transmitted pulse - phase
phase_sim_final = srwlib.array('d', [0]*wfr_sim_final.mesh.nx*wfr_sim_final.mesh.ny) # "flat" array to take 2D phase data
srwl.CalcIntFromElecField(phase_sim_final, wfr_sim_final, 0, 4, 3, wfr_sim_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping data from flat to 2D array
phase_2d_sim_final = np.unwrap(np.unwrap(np.array(phase_sim_final).reshape((wfr_sim_final.mesh.nx, wfr_sim_final.mesh.ny), order='C').astype(np.float64),axis=0),axis=1)

fig = plt.figure()
plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), phase_2d_sim_final, cmap=plt.cm.viridis, shading='auto') #np.unwrap(phase_2d_sim_final)
plt.colorbar()
plt.ylabel(r'Vertical Position [mm]')#[mm]')
plt.xlabel(r'Horizontal Position [mm]')#[mm]')
plt.title('Phase after Drift (no gain)')

***
#### Post Propagation Plots

In [ ]:
e_total = LP.extract_total_2d_elec_fields()
intens_2d_sim_final = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)
    
# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))#(6,3.6))
ax = fig.gca()
plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), intens_2d_sim_final, cmap=plt.cm.viridis, shading='auto') #viridis
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (propagation with gain)')#'E$_{im}$ (radial n2, no gain)')#

In [ ]:
# # transmitted pulse - phase
# phase_sim_final = srwlib.array('d', [0]*wfr_sim_final.mesh.nx*wfr_sim_final.mesh.ny) # "flat" array to take 2D phase data
# srwl.CalcIntFromElecField(phase_sim_final, wfr_sim_final, 0, 4, 3, wfr_sim_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# # Reshaping data from flat to 2D array
# phase_2d_sim_final = np.array(phase_sim_final).reshape((wfr_sim_final.mesh.nx, wfr_sim_final.mesh.ny), order='C').astype(np.float64)

# with plt.style.context(('seaborn-poster')):
#     fig = plt.figure(figsize=(10,6))#(6,3.6))
#     ax = fig.gca()
#     plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), phase_2d_sim_final+1.93, cmap=plt.cm.viridis, shading='auto') #np.unwrap(phase_2d_sim_final)
#     plt.colorbar()
#     # plt.xlim(0,2)
#     # plt.ylim(0,2)
#     ax.set_ylabel(r'Vertical Position [mm]')#[mm]')
#     ax.set_xlabel(r'Horizontal Position [mm]')#[mm]')
#     ax.set_title('Phase (radial n2, no gain) +1.93')

***
## Compare meshes

In [ ]:
# # compare initial and final phase mesh density and range
# print('Phase mesh analysis:')
# print('initial mesh shape: %s , %s' %(np.shape(phase_2d_exp_init)[0], np.shape(phase_2d_exp_init)[1]))
# print('initial horizontal range: %g [mm]' %(np.max(x_exp_init)*(1e3) - np.min(x_exp_init)*(1e3)))
# print('initial vertical range: %g [mm]' %(np.max(y_exp_init)*(1e3) - np.min(y_exp_init)*(1e3)))
# print('final phase mesh shape: %s, %s' %(np.shape(phase_2d_sim_final)[0], np.shape(phase_2d_sim_final)[1]))
# print('final horizontal range: %g [mm]' %(np.max(x_sim_final)*(1e3) - np.min(x_sim_final)*(1e3)))
# print('final vertical range: %g [mm]' %(np.max(y_sim_final)*(1e3) - np.min(y_sim_final)*(1e3)))

# Plot of Intensity Gain

In [ ]:
# Plot difference in intensities

diff_intensity_2D_sim = intens_2d_sim_final - intensity_2D_exp_init

# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))
ax = fig.gca()
plt.pcolormesh(y_sim_final*(1e3), x_sim_final*(1e3), diff_intensity_2D_sim, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (simulation gain minus pump-off data)')

In [ ]:
# print(np.max(intens_2d_sim_final)-np.max(intensity_2D_exp_init))

In [ ]:
prop_n_photons = 0
for laser_index_i in np.arange(LP.nslice):
        prop_n_photons += np.sum(LP.slice[laser_index_i].n_photons_2d.mesh)

# print(prop_n_photons -n_photons_before)
print('Initial number: ', n_photons_before)
print('Number from array: ',prop_n_photons)

In [ ]:
total = 0

for laser_index_i in np.arange(LP.nslice):
    
    dx = (LP.slice[laser_index_i].wfr.mesh.xFin - LP.slice[laser_index_i].wfr.mesh.xStart)/LP.slice[laser_index_i].wfr.mesh.nx
    dy = (LP.slice[laser_index_i].wfr.mesh.yFin - LP.slice[laser_index_i].wfr.mesh.yStart)/LP.slice[laser_index_i].wfr.mesh.ny
    cell_area = dx * dy
    
    intens_2d = srwutil.calc_int_from_elec(LP.slice[laser_index_i].wfr) #extract 2d intensity
    efield_abs_sqrd_2d = np.sqrt(const.mu_0 / const.epsilon_0) * 2.0 * intens_2d # [V^2/m^2]
    
    end1 = (LP.slice[laser_index_i]._pulse_pos -0.5*LP.slice[laser_index_i].ds) /(np.sqrt(2.0) *LP.slice[laser_index_i].sig_s)
    end2 = (LP.slice[laser_index_i]._pulse_pos +0.5*LP.slice[laser_index_i].ds) /(np.sqrt(2.0) *LP.slice[laser_index_i].sig_s)
    energy_2d = cell_area *(const.epsilon_0 /2.0) \
                *(efield_abs_sqrd_2d /np.exp(-LP.slice[laser_index_i]._pulse_pos**2.0/(np.sqrt(2.0) *LP.slice[laser_index_i].sig_s)**2.0)) \
                *((np.sqrt(np.pi)/2.0) *(np.sqrt(2.0) *LP.slice[laser_index_i].sig_s) *(special.erf(end2) -special.erf(end1)))
    photon_e = LP.slice[laser_index_i].photon_e_ev * const.e
    n_photons_2d = energy_2d / photon_e
    total += np.sum(n_photons_2d)

print('Number from fields: ',total)

***
# Plots of peak timing and off-peak timing (assuming same laser params as pump_off case)

In [ ]:
files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_peak_timing.txt'), #_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_peak_timing.txt'), #_off_peak_timing.txt'), #_pump_off.txt')
)

peakPulse = pulse.LaserPulse(params, files)
wfr_exp_final = peakPulse.slice[0].wfr

x_exp_final=np.linspace(wfr_exp_final.mesh.xStart,wfr_exp_final.mesh.xFin,wfr_exp_final.mesh.nx)
y_exp_final=np.linspace(wfr_exp_final.mesh.yStart,wfr_exp_final.mesh.yFin,wfr_exp_final.mesh.ny)

In [ ]:
e_total = peakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))
ax = fig.gca()
plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (peak timing)')

In [ ]:
# Extract phase from experimental post-prop wavefront
phase_exp_final = srwlib.array('d', [0]*wfr_exp_final.mesh.nx*wfr_exp_final.mesh.ny) # "flat" array to take 2D data
srwl.CalcIntFromElecField(phase_exp_final, wfr_exp_final, 0, 4, 3, wfr_exp_final.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping phase data from flat to 2D array
phase_2d_exp_final = np.unwrap(np.unwrap(np.array(phase_exp_final).reshape((wfr_exp_final.mesh.nx, wfr_exp_final.mesh.ny), order='C').astype(np.float64),axis=0),axis=1)

# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))#(6,3.6))
ax = fig.gca()
plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), phase_2d_exp_final, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_title('Phase (peak timing)')

________
With gain on, n1 = 1, n2 = 0, should see gain in peak intensity (gain) that matches the difference between two maximum intensities for peak and off peak timing

In [ ]:
files = PKDict(
    meta = os.path.join(package_data_dir, 'wfs_meta.dat'),
    ccd = os.path.join(package_data_dir, 'ccd_off_peak_timing.txt'), #_pump_off.txt'),
    wfs = os.path.join(package_data_dir, 'wfs_off_peak_timing.txt'), #_pump_off.txt')
)

offpeakPulse = pulse.LaserPulse(params, files)
wfr_exp_final_op = offpeakPulse.slice[0].wfr

# _op = _off_peak

x_exp_final_op=np.linspace(wfr_exp_final_op.mesh.xStart,wfr_exp_final_op.mesh.xFin,wfr_exp_final_op.mesh.nx)
y_exp_final_op=np.linspace(wfr_exp_final_op.mesh.yStart,wfr_exp_final_op.mesh.yFin,wfr_exp_final_op.mesh.ny)

In [ ]:
e_total = offpeakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final_op = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

#with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))
ax = fig.gca()
plt.pcolormesh(y_exp_final_op*(1e3), x_exp_final_op*(1e3), intensity_2D_exp_final_op, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (off-peak timing)')

In [ ]:
# Plot difference in intensities

diff_intensity_2D_exp_final = intensity_2D_exp_final - intensity_2D_exp_final_op

# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))
ax = fig.gca()
plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), diff_intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Intensity (peak timing minus off-peak timing)')

In [ ]:
# print(np.max(intensity_2D_exp_final) - np.max(intensity_2D_exp_final_op))

In [ ]:
# peak_n_photons = 0
# for laser_index_i in np.arange(peakPulse.nslice):
#         peak_n_photons += np.sum(peakPulse.slice[laser_index_i].n_photons_2d.mesh)

# offpeak_n_photons = 0
# for laser_index_i in np.arange(offpeakPulse.nslice):
#         offpeak_n_photons += np.sum(offpeakPulse.slice[laser_index_i].n_photons_2d.mesh)

# print(peak_n_photons -offpeak_n_photons)

In [ ]:
# Extract phase from experimental post-prop wavefront
phase_exp_final = srwlib.array('d', [0]*wfr_exp_final_op.mesh.nx*wfr_exp_final_op.mesh.ny) # "flat" array to take 2D data
srwl.CalcIntFromElecField(phase_exp_final, wfr_exp_final_op, 0, 4, 3, wfr_exp_final_op.mesh.eStart, 0, 0) #extracts the phase; must use double precision

# Reshaping phase data from flat to 2D array
phase_2d_op = np.unwrap(np.unwrap(np.array(phase_exp_final).reshape((wfr_exp_final_op.mesh.nx, wfr_exp_final_op.mesh.ny), order='C').astype(np.float64),axis=0),axis=1)

# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))#(6,3.6))
ax = fig.gca()
plt.pcolormesh(x_exp_final_op*(1e3), y_exp_final_op*(1e3), phase_2d_op, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_title('Phase (off-peak timing)')

In [ ]:
compare = phase_2d_exp_final #phase_2d_exp_final -phase_2d_op #phase_2d_op #
sim_phase = phase_2d_sim_final #phase_2d_sim_final - phase_2d_exp_init #

scale_factor = np.max(compare[:,int(len(y_exp_final)/2.0)]) -np.max(np.unwrap(sim_phase[:,int(len(y_sim_final)/2.0)]))
#print(scale_factor)
# with plt.style.context(('seaborn-poster')):
fig = plt.figure()#figsize=(10,6))#(6,3.6))#
ax = fig.gca()
plt.plot(x_exp_final*(1e3), compare[:,int(len(y_exp_final)/2.0)], 'r', label='Peak timing minus')# off-peak timing')
plt.plot((x_sim_final)*(1e3), scale_factor +sim_phase[:,int(len(y_sim_final)/2.0)], '--k', label='Propagation with gain +{:.3}'.format(scale_factor))
# plt.plot((x_sim_final)*(1e3), np.unwrap(sim_phase[:,int(len(y_sim_final)/2.0)],axis=0), '--k', label='Gain calc minus pump-off data')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax.set_ylabel(r'Phase Lineout')#[mm]')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Phase: Simulation vs Experiment')

#+np.diff(x_sim_final)[0]/2.0

In [ ]:

# scale_factor = np.max(phase_2d_exp_final[:,int(len(x_exp_final)/2.0)])/np.max(-1.0 *phase_2d_sim_final[:,int(len(x_sim_final)/2.0)])
# print(scale_factor)
# with plt.style.context(('seaborn-poster')):
#     fig = plt.figure(figsize=(10,6))#(6,3.6))#
#     ax = fig.gca()
#     plt.plot(y_exp_final*(1e3), phase_2d_exp_final[int(len(x_exp_final)/2.0),:], 'r', label='Peak timing data')
#     plt.plot(y_sim_final*(1e3), -1.0 *phase_2d_sim_final[int(len(x_sim_final)/2.0),:], 'k', label='Propagation with gain')
#     #plt.plot(y_sim_final*(1e3), -10.0 *phase_2d_sim_final[int(len(x_sim_final)/2.0),:], ':k', label='PwG x10')
#     plt.plot(y_sim_final*(1e3), -1.0 *scale_factor *phase_2d_sim_final[int(len(x_sim_final)/2.0),:], '--k', label='PwG x16.98')
#     fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
#     ax.set_ylabel(r'Phase Lineout')#[mm]')
#     ax.set_xlabel(r'Vertical Position [mm]')
#     ax.set_title('Phase: Simulation vs Experiment')

In [ ]:

fig = plt.figure()#figsize=(10,6))#(6,3.6))
ax = fig.gca()
plt.pcolormesh(x_exp_final_op*(1e3), y_exp_final_op*(1e3), compare, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_title('Phase (peak timing)')

fig = plt.figure()#figsize=(10,6))#(6,3.6))
ax = fig.gca()
plt.pcolormesh(x_exp_final_op*(1e3), y_exp_final_op*(1e3), scale_factor +sim_phase, cmap=plt.cm.viridis, shading='auto')
plt.colorbar()
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_ylabel(r'Vertical Position [mm]')
ax.set_title('Phase (propagation with gain +{:.3})'.format(scale_factor))